In [15]:
try:
    import recirq
except ImportError:
    !pip install --quiet git+https://github.com/krainey1/rerecirq

In [16]:
import numpy as np
import cirq

from recirq.hfvqe.gradient_hf import rhf_func_generator
#from recirq.hfvqe.opdm_functionals import OpdmFunctional ~ we are going rework this in here
from recirq.hfvqe.analysis import (
    compute_opdm, mcweeny_purification,
    resample_opdm, fidelity_witness,
    fidelity)
from recirq.hfvqe.third_party.higham import fixed_trace_positive_projection
from recirq.hfvqe.molecular_example import make_h6_1_3

In [17]:
rhf_objective, molecule, parameters, obi, tbi = make_h6_1_3()
ansatz, energy, gradient = rhf_func_generator(rhf_objective)

# settings for quantum resources ~ keep set up for testing
qubits = [cirq.GridQubit(0, x) for x in range(molecule.n_orbitals)]
sampler = cirq.Simulator(dtype=np.complex128)  # this can be a QuantumEngine


Optimization terminated successfully.
         Current function value: -2.924060
         Iterations: 7
         Function evaluations: 15
         Gradient evaluations: 15


In [18]:
#code needed for conversion, adjusted to fix bad register name issue
import cmath
import re
from logging import warning
from typing import Any, cast

import cirq_google
from sympy import Basic, Symbol, pi

import cirq.ops
from cirq.devices import GridQubit, LineQubit
from pytket.architecture import Architecture
from pytket.circuit import Bit, Circuit, Node, OpType, Qubit

# For translating cirq circuits to tket circuits
cirq_common = cirq.ops.common_gates
cirq_pauli = cirq.ops.pauli_gates

cirq_CH = cirq_common.H.controlled(1)

# map cirq common gates to pytket gates
_cirq2ops_mapping = {
    cirq_common.CNOT: OpType.CX,
    cirq_common.H: OpType.H,
    cirq_common.MeasurementGate: OpType.Measure,
    cirq_common.XPowGate: OpType.Rx,
    cirq_common.YPowGate: OpType.Ry,
    cirq_common.ZPowGate: OpType.Rz,
    cirq_common.XPowGate(exponent=0.5): OpType.V,
    cirq_common.XPowGate(exponent=-0.5): OpType.Vdg,
    cirq_common.S: OpType.S,
    cirq_common.SWAP: OpType.SWAP,
    cirq_common.T: OpType.T,
    cirq_pauli.X: OpType.X,
    cirq_pauli.Y: OpType.Y,
    cirq_pauli.Z: OpType.Z,
    cirq.ops.I: OpType.noop,
    cirq_common.CZPowGate: OpType.CU1,
    cirq_common.CZ: OpType.CZ,
    cirq_CH: OpType.CH,
    cirq.ops.CSwapGate: OpType.CSWAP,
    cirq_common.ISwapPowGate: OpType.ISWAP,
    cirq_common.ISWAP: OpType.ISWAPMax,
    cirq.ops.FSimGate: OpType.FSim,
    cirq_google.SYC: OpType.Sycamore,
    cirq.ops.parity_gates.ZZPowGate: OpType.ZZPhase,
    cirq.ops.parity_gates.XXPowGate: OpType.XXPhase,
    cirq.ops.parity_gates.YYPowGate: OpType.YYPhase,
    cirq.ops.PhasedXPowGate: OpType.PhasedX,
    cirq.ops.PhasedISwapPowGate: OpType.PhasedISWAP,
    cirq.ops.common_channels.ResetChannel: OpType.Reset,
}
# reverse mapping for convenience
_ops2cirq_mapping: dict = dict((item[1], item[0]) for item in _cirq2ops_mapping.items())  # noqa: C402
# spot special rotation gates
_constant_gates = (
    cirq_common.CNOT,
    cirq_common.H,
    cirq_common.S,
    cirq_common.SWAP,
    cirq_common.T,
    cirq_pauli.X,
    cirq_pauli.Y,
    cirq_pauli.Z,
    cirq_common.CZ,
    cirq_CH,
    cirq_common.ISWAP,
    cirq_google.SYC,
    cirq.ops.I,
)

_radian_gates = (
    cirq_common.Rx,
    cirq_common.Ry,
    cirq_common.Rz,
)
_cirq2ops_radians_mapping = {
    cirq_common.Rx: OpType.Rx,
    cirq_common.Ry: OpType.Ry,
    cirq_common.Rz: OpType.Rz,
}


def cirq_to_tk(circuit: cirq.circuits.Circuit) -> Circuit:  # noqa: PLR0912, PLR0915
    """Converts a Cirq :py:class:`Circuit` to a tket :py:class:`Circuit` object.

    :param circuit: The input Cirq :py:class:`Circuit`

    :raises NotImplementedError: If the input contains a Cirq :py:class:`Circuit`
        operation which is not yet supported by pytket

    :return: The tket :py:class:`Circuit` corresponding to the input circuit
    """
    tkcirc = Circuit()
    qmap = {}
    i = 0
    for qb in circuit.all_qubits():
        if isinstance(qb, LineQubit):
            uid = Qubit("q", qb.x)
        elif isinstance(qb, GridQubit):
            uid = Qubit("g", qb.row, qb.col)
        elif isinstance(qb, cirq.ops.NamedQubit):
            uid = Qubit(qb.name)
        else:
            raise NotImplementedError("Cannot convert qubits of type " + str(type(qb)))
        tkcirc.add_qubit(uid)
        qmap.update({qb: uid})
    for moment in circuit:
        for op in moment.operations:
            gate = op.gate
            gatetype = type(gate)
            qb_lst = [qmap[q] for q in op.qubits]
            if isinstance(gate, cirq.ops.global_phase_op.GlobalPhaseGate):
                tkcirc.add_phase(cmath.phase(gate.coefficient) / pi)
                continue
            if isinstance(gate, cirq_common.HPowGate) and gate.exponent == 1:
                gate = cirq_common.H
            elif (
                gatetype == cirq_common.CNotPowGate
                and cast("cirq_common.CNotPowGate", gate).exponent == 1
            ):
                gate = cirq_common.CNOT
            elif (
                gatetype == cirq_pauli._PauliX  # noqa: SLF001
                and cast("cirq_pauli._PauliX", gate).exponent == 1  # noqa: SLF001
            ):
                gate = cirq_pauli.X
            elif (
                gatetype == cirq_pauli._PauliY  # noqa: SLF001
                and cast("cirq_pauli._PauliY", gate).exponent == 1  # noqa: SLF001
            ):
                gate = cirq_pauli.Y
            elif (
                gatetype == cirq_pauli._PauliZ  # noqa: SLF001
                and cast("cirq_pauli._PauliZ", gate).exponent == 1  # noqa: SLF001
            ):
                gate = cirq_pauli.Z

            apply_in_parallel = False
            if isinstance(gate, cirq.ops.ParallelGate):
                if gate.num_copies != len(qb_lst):
                    raise NotImplementedError(
                        "ParallelGate parameters defined incorrectly."
                    )
                gate = gate.sub_gate
                gatetype = type(gate)
                apply_in_parallel = True

            if gate in _constant_gates:
                try:
                    optype = _cirq2ops_mapping[gate]
                except KeyError as error:
                    raise NotImplementedError(
                        "Operation not supported by tket: " + str(op.gate)
                    ) from error
                params: list[float | Basic | Symbol] = []
            elif isinstance(gate, cirq.ops.common_channels.ResetChannel):
                optype = OpType.Reset
                params = []
            elif gatetype in _radian_gates:
                try:
                    optype = _cirq2ops_radians_mapping[
                        cast("type[cirq.ops.EigenGate]", gatetype)
                    ]
                except KeyError as error:
                    raise NotImplementedError(
                        "Operation not supported by tket: " + str(op.gate)
                    ) from error
                params = [gate._rads / pi]  # type: ignore  # noqa: SLF001
            elif isinstance(gate, cirq_common.MeasurementGate):
                # Adding "_b" to the bit uid since for cirq.NamedQubit,
                # the gate.key is equal to the qubit id (the qubit name)
                bitid = Bit("c", [i]) #no longer produces bad register name q(0,0)_b, etc. -> c[0]...
                i += 1
                tkcirc.add_bit(bitid)
                assert len(qb_lst) == 1
                tkcirc.Measure(qb_lst[0], bitid)
                continue
            elif isinstance(gate, cirq.ops.PhasedXPowGate):
                optype = OpType.PhasedX
                pe = gate.phase_exponent
                params = [gate.exponent, pe]
            elif isinstance(gate, cirq.ops.FSimGate):
                optype = OpType.FSim
                params = [gate.theta / pi, gate.phi / pi]
            elif isinstance(gate, cirq.ops.PhasedISwapPowGate):
                optype = OpType.PhasedISWAP
                params = [gate.phase_exponent, gate.exponent]
            else:
                try:
                    optype = _cirq2ops_mapping[gatetype]
                    params = [cast("Any", gate).exponent]
                except (KeyError, AttributeError) as error:
                    raise NotImplementedError(
                        "Operation not supported by tket: " + str(op.gate)
                    ) from error
            if apply_in_parallel:
                for qbit in qb_lst:
                    tkcirc.add_gate(optype, params, [qbit])
            else:
                tkcirc.add_gate(optype, params, qb_lst)
    return tkcirc


In [22]:
#redefine the opdm class
from typing import List, Optional
import numpy as np
import scipy as sp
import pandas as pd
import pytket
from pytket import Bit
from pytket.utils.distribution import ProbabilityDistribution
import qnexus as qnx
import cirq
import datetime
import recirq.hfvqe.circuits as ccc
import recirq.hfvqe.analysis as cca
import recirq.hfvqe.util as ccu
from recirq.hfvqe.objective import rhf_params_to_matrix
class OpdmFunctional():  # testpragma: no cover

    def __init__(self,
                 qubits: List[cirq.Qid],
                 sampler: cirq.Sampler,
                 constant: float,
                 one_body_integrals: np.ndarray,
                 two_body_integrals: np.ndarray,
                 num_electrons: int,
                 num_samples: Optional[int] = 25, #was 250000 shots, running smaller # 25 for debugging
                 post_selection: Optional[bool] = True,
                 purification: Optional[bool] = True,
                 clean_xxyy: Optional[bool] = True,
                 verbose: Optional[bool] = False):
        self.qubits = qubits
        self.constant = constant
        self.one_body_integrals = one_body_integrals
        self.two_body_integrals = two_body_integrals
        self.num_electrons = num_electrons
        self.sampler = sampler
        self.num_samples = num_samples
        self.post_selection = post_selection
        self.purification = purification
        self.clean_xxyy = clean_xxyy
        self.num_qubits = len(self.qubits)
        self.verbose = verbose  # type: bool

        self._last_noisy_opdm = None

        norbs = len(self.qubits)
        self.occ = list(range(num_electrons))
        self.virt = list(range(num_electrons, norbs))
        self.nocc = len(self.occ)
        self.nvirt = len(self.virt)
        self.clean_xxyy = clean_xxyy

    def calculate_data(self, parameters):
        #qnexus project setup
        project = qnx.projects.get_or_create(name="google_hf_rework")
        config = qnx.QuantinuumConfig(device_name="H2-Emulator") #H2 and H1 switched here
        qnx.context.set_active_project(project)
        
        if len(parameters.shape) == 2:  # testpragma: no cover
            u = parameters
        else:
            u = sp.linalg.expm(
                rhf_params_to_matrix(parameters,
                                     len(self.qubits),
                                     occ=self.occ,
                                     virt=self.virt))

        circuits = ccc.generate_circuits_from_params_or_u(
            self.qubits,
            u,
            self.num_electrons,
            occ=self.occ,
            virt=self.virt,
            clean_ryxxy=self.clean_xxyy)
        circuits_dict = ccc.circuits_with_measurements(
            self.qubits, circuits, clean_xxyy=self.clean_xxyy)

        # Take data
        data_dict = {
            'z': {},
            'xy_even': {},
            'xy_odd': {},
            'qubits': [str(q) for q in self.qubits],
            'qubit_permutations': ccu.generate_permutations(len(self.qubits)),
            'circuits': circuits,
            'circuits_with_measurement': circuits_dict
        }
        for measure_type in circuits_dict.keys():
            circuits = circuits_dict[measure_type]
            for circuit_index in circuits.keys():
                jobname_suffix = datetime.datetime.now().strftime("%Y_%m_%d-%H-%M-%S") #allow for new jobname on each iteration, fix failed resource allocate
                circuit = circuits[circuit_index]
                if self.verbose:  # testpragma: no cover
                    print(circuit.to_text_diagram(transpose=True))
                #data = self.sampler.run(circuit, repetitions=self.num_samples)
                #instead want to convert the circuit
                #print(data)
                #print(data.data) 
                circuit = cirq_to_tk(circuit) #circuit conversion
                #print(circuit.qubits) sanity checking
                #print(circuit.bits)
                
                #upload circuit, compile it, execute it for x amount of shots
                ref = qnx.circuits.upload(circuit=circuit, name=f"test circuit {jobname_suffix}")
                compile_circuit = qnx.start_compile_job(
                    programs = [ref],
                    backend_config=config,
                    name=f"test circuit {jobname_suffix}",
                )
                qnx.jobs.wait_for(compile_circuit)
                compile_circuit = [item.get_output() for item in qnx.jobs.results(compile_circuit)]
                exec_circuit = qnx.start_execute_job(
                    programs=compile_circuit,
                    n_shots=[self.num_samples],
                    backend_config=config,
                    name=f"execution circuit {jobname_suffix}",
                )
                qnx.jobs.wait_for(exec_circuit)
                ref_result = qnx.jobs.results(exec_circuit)[0]
                backend_result = ref_result.download_result()
                #print(backend_result.get_counts()) #returns a Counter object
                counter = backend_result.get_counts()
                #build a pandas dataframe like cirq does for the results
                rows = []
                for key, count in counter.items():
                    for _ in range(count):
                        rows.append(key)
                
                #Converting to DataFrame
                df = pd.DataFrame(rows)
                df = df.rename(columns={0: "q(0, 0)", 1: "q(0, 1)", 2: "q(0, 2)", 3: "q(0, 3)", 4: "q(0, 4)", 5: "q(0, 5)"})
                print(df)

                #adjusted below to switch out cirq dataframe for our dataframe of data from H1/H2
                if self.post_selection:
                    # PostSelect the data
                    good_indices = \
                        np.where(np.sum(np.array(df), axis=1) ==
                                            self.num_electrons)[0]
                    good_data = df[df.index.isin(good_indices)]
                    data_dict[measure_type][circuit_index] = good_data
                else:  # testpragma: no cover
                    data_dict[measure_type][circuit_index] = df
        return data_dict

    def calculate_rdm(self, parameters):
        data_dict = self.calculate_data(parameters)
        opdm, opdm_var_dict = cca.compute_opdm(data_dict, return_variance=True)
        if self.purification:
            self._last_noisy_opdm = opdm
            opdm = cca.mcweeny_purification(opdm)

        return opdm, opdm_var_dict

    def energy_from_opdm(self, opdm):
        return cca.energy_from_opdm(opdm, self.constant,
                                    self.one_body_integrals,
                                    self.two_body_integrals)


In [23]:
#build interface
opdm_func = OpdmFunctional(qubits=qubits,
                           sampler=sampler,
                           constant=molecule.nuclear_repulsion,
                           one_body_integrals=obi,
                           two_body_integrals=tbi,
                           # only simulate spin-up electrons:
                           num_electrons=molecule.n_electrons // 2,
                           clean_xxyy=True,
                           purification=True
                           )

In [24]:
measurement_data = opdm_func.calculate_data(parameters) #execution time (+-+)

    q(0, 0)  q(0, 1)  q(0, 2)  q(0, 3)  q(0, 4)  q(0, 5)
0         1        0        0        1        1        0
1         1        0        1        0        0        1
2         1        0        1        1        0        0
3         1        0        1        1        0        0
4         1        0        1        1        0        0
5         1        0        1        1        0        0
6         1        0        1        1        0        0
7         1        0        1        1        0        0
8         1        1        0        0        1        0
9         1        1        0        0        1        0
10        1        1        0        0        1        0
11        1        1        0        0        1        0
12        1        1        0        0        1        0
13        1        1        1        0        0        0
14        1        1        1        0        0        0
15        1        1        1        0        0        0
16        1        1        1  

In [25]:
opdm, var_dict = compute_opdm(measurement_data, return_variance=True)
opdm_pure = mcweeny_purification(opdm)

# 3.
raw_energies = []
raw_fidelity_witness = []
purified_eneriges = []
purified_fidelity_witness = []
purified_fidelity = []
true_unitary = ansatz(parameters)
nocc = molecule.n_electrons // 2
nvirt = molecule.n_orbitals - nocc
initial_fock_state = [1] * nocc + [0] * nvirt

# 1000 repetitions of the measurement
for _ in range(1000):
    new_opdm = resample_opdm(opdm, var_dict)
    raw_energies.append(opdm_func.energy_from_opdm(new_opdm))
    raw_fidelity_witness.append(
        fidelity_witness(target_unitary=true_unitary,
                         omega=initial_fock_state,
                         measured_opdm=new_opdm)
    )
    # fix positivity and trace of sampled 1-RDM if strictly outside
    # feasible set
    w, v = np.linalg.eigh(new_opdm)
    if len(np.where(w < 0)[0]) > 0:
        new_opdm = fixed_trace_positive_projection(new_opdm, nocc)

    new_opdm_pure = mcweeny_purification(new_opdm)
    purified_eneriges.append(opdm_func.energy_from_opdm(new_opdm_pure))
    purified_fidelity_witness.append(
        fidelity_witness(target_unitary=true_unitary,
                         omega=initial_fock_state,
                         measured_opdm=new_opdm_pure)
    )
    purified_fidelity.append(
        fidelity(target_unitary=true_unitary,
                 measured_opdm=new_opdm_pure)
    )
print("Canonical Hartree-Fock energy ", molecule.hf_energy)
print("True energy ", energy(parameters))
print("Raw energy ", opdm_func.energy_from_opdm(opdm),
      "+- ", np.std(raw_energies))
print("Raw fidelity witness ", np.mean(raw_fidelity_witness).real,
      "+- ", np.std(raw_fidelity_witness))
print("purified energy ", opdm_func.energy_from_opdm(opdm_pure),
      "+- ", np.std(purified_eneriges))
print("Purified fidelity witness ", np.mean(purified_fidelity_witness).real,
      "+- ", np.std(purified_fidelity_witness))
print("Purified fidelity ", np.mean(purified_fidelity).real,
      "+- ", np.std(purified_fidelity))

Canonical Hartree-Fock energy  -2.9240604849733085
True energy  -2.9240604849722285
Raw energy  -2.5874648056361114 +-  0.16654750557144302
Raw fidelity witness  0.48192994415675083 +-  0.24694509438641196
purified energy  -2.862957737406696 +-  0.08661713431964217
Purified fidelity witness  0.7547921072336073 +-  0.12006395917472262
Purified fidelity  0.881010003244973 +-  0.05618567519856153
